In [1]:
# !pip install pandas tqdm transformers accelerate bitsandbytes

In [2]:
import re
import os
import pandas as pd
from tqdm import tqdm

import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [3]:
test = pd.read_csv('../data/test.csv')
test

,ID,Question
0,TEST_000,금융산업의 이해와 관련하여 금융투자업의 구분에 해당하지 않는 것은?\n1 소비자금융...
1,TEST_001,위험 관리 계획 수립 시 고려해야 할 요소로 적절하지 않은 것은?\n1 수행인력\n...
2,TEST_002,관리체계 수립 및 운영'의 '정책 수립' 단계에서 가장 중요한 요소는 무엇인가?\n...
3,TEST_003,재해 복구 계획 수립 시 고려해야 할 요소로 옳지 않은 것은?\n1 복구 절차 수립...
4,TEST_004,트로이 목마(Trojan) 기반 원격제어 악성코드(RAT)의 특징과 주요 탐지 지표...
...,...,...
510,TEST_510,"""정보보호최고책임자""의 임명에 관한 설명으로 옳지 않은 것은?\n1 정보보호최고책임..."
511,TEST_511,IPv6 주소 체계의 주요 특징으로 옳지 않은 것은?\n1 NAT 필요성 감소\n2...
512,TEST_512,하이브리드 위협에 대한 설명으로 가장 적절한 것은?\n1 사이버 공간에서만 발생하는...
513,TEST_513,전자금융거래법의 주요 목적 중 하나는 무엇인가?\n1 전자금융거래의 비대면성 강화\...


In [4]:
test = test.iloc[[492, 446, 362, 111, 121, 162, 434, 4, 7, 82, 101, 102, 126, 131, 182, 458], :]
# test = test.iloc[:20, :]1

In [5]:
# 객관식 여부 판단 함수
def is_multiple_choice(question_text):
    """
    객관식 여부를 판단: 2개 이상의 숫자 선택지가 줄 단위로 존재할 경우 객관식으로 간주
    """
    lines = question_text.strip().split("\n")
    option_count = sum(bool(re.match(r"^\s*[1-9][0-9]?\s", line)) for line in lines)
    return option_count >= 2


# 질문과 선택지 분리 함수
def extract_question_and_choices(full_text):
    """
    전체 질문 문자열에서 질문 본문과 선택지 리스트를 분리
    """
    lines = full_text.strip().split("\n")
    q_lines = []
    options = []

    for line in lines:
        if re.match(r"^\s*[1-9][0-9]?\s", line):
            options.append(line.strip())
        else:
            q_lines.append(line.strip())

    question = " ".join(q_lines)
    return question, options

In [6]:
# 프롬프트 생성기
def make_prompt_auto(text):
    if is_multiple_choice(text):
        question, options = extract_question_and_choices(text)
        prompt = (
                "당신은 금융보안 전문가입니다.\n"
                "아래 질문에 대해 적절한 **정답 선택지 번호만 출력**하세요.\n\n"
                f"질문: {question}\n"
                "선택지:\n"
                f"{chr(10).join(options)}\n\n"
                "답변:"
                )
    else:
        prompt = (
                "당신은 금융보안 전문가입니다.\n"
                # "아래 주관식 질문에 대해 정확하고 간략한 설명을 작성하세요.\n\n"
                "아래 질문에 대해 정답의 핵심 키워드와 의미를 모두 포함하여 3문장 이내로 간결하게 답변하세요. 군더더기 없이 요점만 명확하게 작성하세요.\n\n"
                f"질문: {text}\n\n"
                "답변:"
                )
    return prompt

In [7]:
# -*- coding: utf-8 -*-
# 멀티법령 RAG 전처리/인덱싱 (PIPA, 신용정보법, 전자서명법, 정보통신망법)

import os, re, json, math
from dataclasses import dataclass, asdict
from typing import List, Dict, Tuple, Optional

import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

from PyPDF2 import PdfReader
from langchain_community.embeddings import HuggingFaceEmbeddings
import faiss

# ===== 사용자 환경 상수 =====
# LLM_ID = "nlpai-lab/KULLM3"
LLM_ID = "MLP-KTLim/llama-3-Korean-Bllossom-8B"
EMB_MODEL = "jhgan/ko-sroberta-multitask"   # 경량/호환성 위주
# EMB_MODEL = "intfloat/multilingual-e5-base"
CHUNK_TOKENS = 600
CHUNK_OVERLAP = 32
CTX_TOKEN_BUDGET = 1200
TOP_K = 3
SEED = 42
torch.manual_seed(SEED)

# ===== 법령 설정: 파일 경로 + 제거/정규식 패턴 =====
LAW_CONFIG = {
    # 개인정보 보호법
    "pipa": {
        "law_name": "개인정보 보호법",
        "pdf_path": "../data/개인정보 보호법(법률)(제19234호)(20250313).pdf",
        "drop_patterns": [
            r'법제처\s+\d+\s+국가법령정보센터\s*개인정보\s*보호법',
            r'법제처\s+\d+\s+국가법령정보센터',
            r'국가법령정보센터\s*개인정보\s*보호법',
            r'법제처|국가법령정보센터',
            r'<[^>]+>',         # <개정 …>, <신설 …>
            r'\[[^\]]+\]',      # [본조신설 …]
        ],
    },
    # 신용정보의 이용 및 보호에 관한 법률
    "ciupa": {
        "law_name": "신용정보법",
        "pdf_path": "../data/신용정보의 이용 및 보호에 관한 법률(법률)(제20304호)(20240814).pdf",
        "drop_patterns": [
            r'법제처\s+\d+\s+국가법령정보센터\s*신용정보.*법',
            r'법제처|국가법령정보센터',
            r'<[^>]+>', r'\[[^\]]+\]',
        ],
    },
    # 전자서명법
    "es_act": {
        "law_name": "전자서명법",
        "pdf_path": "../data/전자서명법(법률)(제18479호)(20221020).pdf",
        "drop_patterns": [
            r'법제처\s+\d+\s+국가법령정보센터\s*전자서명법',
            r'법제처|국가법령정보센터',
            r'<[^>]+>', r'\[[^\]]+\]',
        ],
    },
    # 정보통신망 이용촉진 및 정보보호 등에 관한 법률
    "icn_act": {
        "law_name": "정보통신망법",
        "pdf_path": "../data/정보통신망 이용촉진 및 정보보호 등에 관한 법률(법률)(제20678호)(20250722).pdf",
        "drop_patterns": [
            r'법제처\s+\d+\s+국가법령정보센터\s*정보통신망.*법',
            r'법제처|국가법령정보센터',
            r'<[^>]+>', r'\[[^\]]+\]',
        ],
    },
    # 전자금융거래법
    "eft_act": {
        "law_name": "전자금융거래법",
        "pdf_path": "../data/전자금융거래법(법률)(제19734호)(20240915).pdf",
        "drop_patterns": [
            r'법제처\s+\d+\s+국가법령정보센터\s*전자금융거래.*법',
            r'법제처|국가법령정보센터',
            r'<[^>]+>', r'\[[^\]]+\]',
        ],
    },
    # 전자금융감독규정
    "rs_act": {
        "law_name": "전자금융감독규정",
        "pdf_path": "../data/전자금융감독규정(금융위원회고시)(제2025-4호)(20250205).pdf",
        "drop_patterns": [
            r'법제처\s+\d+\s+국가법령정보센터\s*전자금융거래.*법',
            r'법제처|국가법령정보센터',
            r'<[^>]+>', r'\[[^\]]+\]',
        ],
    },
    # 자본시장법
    "fis_act": {
        "law_name": "자본시장법",
        "pdf_path": "../data/자본시장과 금융투자업에 관한 법률(법률)(제20718호)(20250722).pdf",
        "drop_patterns": [
            r'법제처\s+\d+\s+국가법령정보센터\s*자본시장.*법',
            r'법제처|국가법령정보센터',
            r'<[^>]+>', r'\[[^\]]+\]',
        ],
    },
}

In [8]:
# ===== 토크나이저: 토큰 길이 계산/청킹 =====
llm_tokenizer = AutoTokenizer.from_pretrained(LLM_ID)
if llm_tokenizer.pad_token is None:
    llm_tokenizer.pad_token = llm_tokenizer.eos_token
llm_tokenizer.padding_side = "right"

def token_len(s: str) -> int:
    return len(llm_tokenizer(s, add_special_tokens=False)["input_ids"])

def split_sentences_ko(text: str) -> List[str]:
    # 고정폭 lookbehind: '다.' 또는 일반 종결부호
    text = re.sub(r'\s+', ' ', text).strip()
    if not text:
        return []
    return re.split(r'(?<=다\.)\s+|(?<=[.?!。！？])\s+', text)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
# ===== 공통 정제 =====
def normalize_common(text: str) -> str:
    # 한자 제거
    text = re.sub(r'[\u4e00-\u9fff]', '', text)
    # circled numbers → (n)
    circled = '①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳'
    for idx, c in enumerate(circled, 1):
        text = text.replace(c, f'({idx})')
    # 공백/빈 괄호 정리
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\(\s*\)', '', text)
    return text

def clean_text_by_config(text: str, drop_patterns: List[str]) -> str:
    for pat in drop_patterns:
        text = re.sub(pat, '', text)
    return normalize_common(text)

# ===== 조문 단위 분리 =====
ARTICLE_HEADER_PATTERN = r'(제\d+조(?:의\d+)?\([^)]+\))'  # 제X조(제목) / 제X조의Y(제목)

def split_articles(raw_text: str) -> List[Tuple[str, str, str]]:
    parts = re.split(ARTICLE_HEADER_PATTERN, raw_text)
    out = []
    for i in range(1, len(parts), 2):
        header = parts[i]
        body = (parts[i+1] if i+1 < len(parts) else "").strip().replace("\n", " ")
        m = re.match(r'(제\d+조(?:의\d+)?)[(]([^)]+)[)]', header)
        if not m:
            continue
        article_id = m.group(1)          # 제xx조 / 제xx조의y
        article_title = m.group(2)       # (제목)
        out.append((article_id, article_title, body))
    return out

def chunk_article(article_body: str, header: str) -> List[str]:
    prefix = header.strip() + "\n"
    sents = split_sentences_ko(article_body) or [article_body]
    chunks, cur, cur_toks = [], [], token_len(prefix)
    for s in sents:
        tl = token_len(s)
        if cur_toks + tl > CHUNK_TOKENS and cur:
            chunks.append(prefix + " ".join(cur))
            # overlap: 마지막 문장 유지
            keep = cur[-1] if CHUNK_OVERLAP > 0 and cur else ""
            cur = [keep] if keep else []
            cur_toks = token_len(prefix) + (token_len(keep) if keep else 0)
        cur.append(s); cur_toks += tl
    if cur:
        chunks.append(prefix + " ".join(cur))
    return chunks


In [10]:
# ===== 데이터 클래스 =====
@dataclass
class LawDoc:
    text: str
    meta: Dict

# ===== 임베딩 =====
embeddings = HuggingFaceEmbeddings(
    model_name=EMB_MODEL,
    model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"},
    encode_kwargs={
        "normalize_embeddings": True,
        "batch_size": 128,
        "convert_to_numpy": True,
        "convert_to_tensor": False
    }
)

# ===== 인덱스 (FAISS HNSW) =====
def build_faiss_hnsw(vectors: np.ndarray, m: int = 32, ef_search: int = 32) -> faiss.IndexHNSWFlat:
    dim = vectors.shape[1]
    idx = faiss.IndexHNSWFlat(dim, m)
    idx.hnsw.efSearch = ef_search
    idx.add(vectors.astype(np.float32))
    return idx

# ===== 파이프라인: 1) 로드 → 2) 정제 → 3) 조문분리 → 4) 청킹 → 5) 임베딩/인덱스
def load_pdf_text(pdf_path: str) -> str:
    reader = PdfReader(pdf_path)
    text = ""
    for p in reader.pages:
        t = p.extract_text() or ""
        text += t + "\n"
    return text

def preprocess_law(law_id: str, cfg: Dict) -> List[LawDoc]:
    raw = load_pdf_text(cfg["pdf_path"])
    cleaned = clean_text_by_config(raw, cfg["drop_patterns"])
    articles = split_articles(cleaned)

    docs: List[LawDoc] = []
    # 시행일 파싱(있으면): [시행 YYYY. M. D.]가 머리말에 있는 경우가 많으나 PDF마다 다름 → 필요 시 별도 파서
    effective_date = None  # 필요 시 추출 로직 추가

    for article_id, title, body in articles:
        header = f'{cfg["law_name"]} {article_id}({title})'
        chunks = chunk_article(body, header)
        for ch in chunks:
            meta = {
                "law_id": law_id,
                "law_name": cfg["law_name"],
                "article_id": article_id,
                "article_title": title,
                "effective_date": effective_date,   # None 가능
                "tok_len": token_len(ch),
                "source_uri": cfg.get("pdf_path"),
                "version": None
            }
            docs.append(LawDoc(text=ch, meta=meta))
    return docs

def build_indices(all_docs: List[LawDoc]):
    # (a) 법령별 인덱스
    per_law_docs: Dict[str, List[LawDoc]] = {}
    for d in all_docs:
        per_law_docs.setdefault(d.meta["law_id"], []).append(d)

    indices = {}
    for law_id, docs in per_law_docs.items():
        mat = np.array(embeddings.embed_documents([d.text for d in docs]), dtype=np.float32)
        indices[f"faiss_hnsw_{law_id}"] = {
            "index": build_faiss_hnsw(mat, m=32, ef_search=32),
            "docs": docs
        }

    # (b) 글로벌 인덱스
    mat_all = np.array(embeddings.embed_documents([d.text for d in all_docs]), dtype=np.float32)
    indices["faiss_hnsw_all"] = {
        "index": build_faiss_hnsw(mat_all, m=32, ef_search=32),
        "docs": all_docs
    }
    return indices

/tmp/ipykernel_1592/3198458753.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [11]:
all_docs: List[LawDoc] = []
for law_id, cfg in LAW_CONFIG.items():
    if not os.path.exists(cfg["pdf_path"]):
        print(f"[WARN] PDF not found: {cfg['pdf_path']}")
        continue
    docs = preprocess_law(law_id, cfg)
    all_docs.extend(docs)
    print(f"[OK] {cfg['law_name']} → chunks: {len(docs)}")

indices = build_indices(all_docs)
print("[OK] built indices:", list(indices.keys()))

[OK] 개인정보 보호법 → chunks: 160
[OK] 신용정보법 → chunks: 125
[OK] 전자서명법 → chunks: 26
[OK] 정보통신망법 → chunks: 113
[OK] 전자금융거래법 → chunks: 78
[OK] 전자금융감독규정 → chunks: 113
[OK] 자본시장법 → chunks: 782
[OK] built indices: ['faiss_hnsw_pipa', 'faiss_hnsw_ciupa', 'faiss_hnsw_es_act', 'faiss_hnsw_icn_act', 'faiss_hnsw_eft_act', 'faiss_hnsw_rs_act', 'faiss_hnsw_fis_act', 'faiss_hnsw_all']


In [12]:
# ===== 라우팅 규칙 =====
ARTICLE_PTRN = re.compile(r"제\d+조(?:의\d+)?")  # 조문 표기
LAW_HINTS = {
    "pipa": ("개인정보", "개인 정보", "개인정보보호법"),
    "ciupa": ("신용정보",  "신용정보법"),
    "es_act": ("전자서명", "전자서명법"),
    "icn_act": ("정보통신망", "통신망", "정보통신망법"),
    "eft_act": ("전자금융", "금융거래", "전자금융거래법"),
    "rs_act": ("전자금융감독", "금융감독", "전자금융감독규정"),
    "fis_act": ("자본", "자본시장", "금융투자", "채권", "자본시장법", "금융투자업", "투자"),
}

def detect_law_id(query: str) -> Optional[str]:
    q = query.lower()
    for law_id, kws in LAW_HINTS.items():
        if any(kw.lower() in q for kw in kws):
            return law_id
    return None

def route_is_domain(query: str) -> bool:
    # 법/금융/보안 도메인 간단 라우터 (검색 여부 판단용)
    domain_kws = ("법", "조(", "과징금", "처벌", "보안", "침해", "금융", "개인정보", "신용정보", "전자서명", "정보통신", "전자금융", "금융감독", "자본", "자본시장", "금융투자", "채권", "투자")
    q = query.lower()
    return any(kw in q for kw in domain_kws) or bool(ARTICLE_PTRN.search(query))

def choose_index(indices: dict, query: str):
    """
    1) 질의에서 법령 단서 -> 해당 법 인덱스 우선
    2) 조문 패턴만 있거나 단서가 불분명 -> 글로벌 인덱스
    3) 아무 단서도 없으면 None (베이스모델 경로)
    """
    law_id = detect_law_id(query)
    if law_id:
        key = f"faiss_hnsw_{law_id}"
        if key in indices:
            return indices[key]  # {"index": ..., "docs": ...}
    # 법령 단서 없지만 도메인성/조문 표기는 있는 경우 글로벌
    if route_is_domain(query) and "faiss_hnsw_all" in indices:
        return indices["faiss_hnsw_all"]
    return None  # 베이스모델 직행


In [13]:
# ===== 검색 with 점수 (해당 인덱스에서) =====
def faiss_search_with_scores_from_index(index_entry: dict, query: str, top_k: int = TOP_K):
    # embeddings는 상위 스코프에서 로드되었다고 가정
    qv = np.array(embeddings.embed_query(query), dtype=np.float32).reshape(1, -1)
    D, I = index_entry["index"].search(qv, top_k)     # L2 거리 (정규화 벡터 가정)
    cos = 1.0 - (D[0] / 2.0)                          # L2 -> cosine
    out = []
    docs = index_entry["docs"]
    for idx, i in enumerate(I[0]):
        ii = int(i)
        if ii >= 0:
            out.append((docs[ii], float(cos[idx])))
    return out  # [(LawDoc, cosine), ...]

# ===== 컨텍스트 패킹 (질문 주신 코드 재사용 + tok_len 캐시) =====
def pack_context(docs_in, token_budget=CTX_TOKEN_BUDGET):
    acc, used = [], 0
    for d in docs_in:
        tl = d.meta.get("tok_len", None)
        if tl is None:
            tl = token_len(d.text); d.meta["tok_len"] = tl
        if used + tl <= token_budget:
            acc.append(d.text); used += tl
        else:
            remain = token_budget - used
            if remain > 50:
                ids = llm_tokenizer(d.text, add_special_tokens=False)["input_ids"][:remain]
                acc.append(llm_tokenizer.decode(ids))
            break
    return "\n\n".join(acc)


In [14]:
# ===== 프롬프트 =====
def chat_prompt(system, user):
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": user},
    ]
    return llm_tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

def build_prompt(query: str, use_context: bool, context) -> Tuple[str, int]:
    """컨텍스트 유무에 따라 간단 프롬프트와 max_len을 반환"""
    # 객관식 질문
    if is_multiple_choice(query):
        question, options = extract_question_and_choices(query)
        if use_context:
            prompt = (
                "아래 컨텍스트를 우선 사용해 정확히 답하세요. 불충분하면 아는 범위에서만 간결히 답하세요.\n\n"
                "아래 질문에 대해 적절한 **정답 선택지 번호만 출력**하세요.\n\n"
                f"=== 컨텍스트 ===\n{context}\n=== 끝 ===\n"
                f"질문: {question}\n"
                "선택지:\n"
                f"{chr(10).join(options)}\n\n"
                "답변:"
            )

            system = "당신은 금융/보안 QA 도우미입니다. 포맷을 엄격히 지키세요."
            prompt = chat_prompt(system, prompt)

            max_new = dynamic_max_new_tokens(query)
            max_len = 3072  # 입력 길이 상한도 줄여 토크나이즈 시간 단축
            return prompt, max_len
        else:
            prompt = (
                "아래 질문에 대해 적절한 **정답 선택지 번호만 출력**하세요.\n\n"
                f"질문: {question}\n"
                "선택지:\n"
                f"{chr(10).join(options)}\n\n"
                "답변:"
            )

            system = "당신은 금융/보안 QA 도우미입니다. 포맷을 엄격히 지키세요."
            prompt = chat_prompt(system, prompt)

            max_new = dynamic_max_new_tokens(query)
            max_len = 2048
            return prompt, max_len
    # 주관식 질문
    else:
        if use_context:
            prompt = (
                "아래 컨텍스트를 우선 사용해 정확히 답하세요. 불충분하면 아는 범위에서만 간결히 답하세요.\n\n"
                f"=== 컨텍스트 ===\n{context}\n=== 끝 ===\n"
                """아래 질문에 대해 **사실에 근거한 간결한 답변**을 작성하세요. 

규칙:
1. 답변은 2~3문장 이내로 작성합니다. 장황한 서론, 결론 문구는 쓰지 않습니다.
2. 불확실할 경우 '알 수 없습니다'라고 답하고 생성을 종료합니다.
3. 특수문자 없이 오로지 한글과 숫자로만 대답합니다.
"""
                f"질문: {query}"
                "답변:"
            )

            system = "당신은 금융/보안 QA 도우미입니다. 포맷을 엄격히 지키세요."
            prompt = chat_prompt(system, prompt)

            max_new = dynamic_max_new_tokens(query)
            max_len = 3072  # 입력 길이 상한도 줄여 토크나이즈 시간 단축
            return prompt, max_len
        else:
            prompt = (
                """아래 질문에 대해 **사실에 근거한 간결한 답변**을 작성하세요. 

규칙:
1. 답변은 2~3문장 이내로 작성합니다. 장황한 서론, 결론 문구는 쓰지 않습니다.
2. 불확실할 경우 '알 수 없습니다'라고 답하고 생성을 종료합니다.
3. 특수문자 없이 오로지 한글과 숫자로만 대답합니다.
"""
                f"질문: {query}"
                "답변:"
            )

            system = "당신은 금융/보안 QA 도우미입니다. 포맷을 엄격히 지키세요."
            prompt = chat_prompt(system, prompt)

            max_new = dynamic_max_new_tokens(query)
            max_len = 2048
            return prompt, max_len


# ===== 생성 토큰 상한 =====
def dynamic_max_new_tokens(question: str) -> int:
    lines = [ln.strip() for ln in question.split("\n") if ln.strip()]
    opt_cnt = sum(bool(re.match(r"^\d+(\s|[.)])", ln)) for ln in lines)
    return 96 if opt_cnt >= 2 else 192

# ===== 메인: 다중 인덱스 기반 텍스트 생성 =====
def generate_answer_with_indices(query: str, indices: dict) -> str:
    """
    indices: build_indices() 반환 구조
    - 라우팅 → 해당 인덱스에서 top-k 검색(점수 포함)
    - 최고 유사도 임계치 미만이면 컨텍스트 없이 베이스모델 생성(Adaptive RAG)
    """
    # 0) 우선, 일반상식/비도메인은 곧장 베이스모델
    if not route_is_domain(query):
        prompt, max_len = build_prompt(query, use_context=False, context=None)
        inputs = llm_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_len, padding=False)
        inputs = {k: v.to(llm_model.device) for k, v in inputs.items()}
        with torch.inference_mode():
            out = llm_model.generate(**inputs,
                                     max_new_tokens=dynamic_max_new_tokens(query),
                                     do_sample=False,
                                     temperature=0.0,
                                     top_p=0.1,
                                     repetition_penalty=1.1,
                                     eos_token_id=llm_tokenizer.eos_token_id,
                                     pad_token_id=llm_tokenizer.pad_token_id,
                                     )
        gen = out[0][inputs["input_ids"].shape[1]:]
        return llm_tokenizer.decode(gen, skip_special_tokens=True).strip()

    # 1) 인덱스 선택
    idx_entry = choose_index(indices, query)

    # 2) 인덱스가 없으면 베이스모델
    if idx_entry is None:
        prompt, max_len = build_prompt(query, use_context=False, context=None)
        inputs = llm_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_len, padding=False)
        inputs = {k: v.to(llm_model.device) for k, v in inputs.items()}
        with torch.inference_mode():
            out = llm_model.generate(**inputs,
                                     max_new_tokens=dynamic_max_new_tokens(query),
                                     do_sample=False,
                                     temperature=0.0,
                                     top_p=0.1,
                                     repetition_penalty=1.1,
                                     eos_token_id=llm_tokenizer.eos_token_id,
                                     pad_token_id=llm_tokenizer.pad_token_id,
                                     )
        gen = out[0][inputs["input_ids"].shape[1]:]
        return llm_tokenizer.decode(gen, skip_special_tokens=True).strip()

    # 3) 선택 인덱스에서 검색 + 점수
    scored = faiss_search_with_scores_from_index(idx_entry, query, top_k=TOP_K)
    best_cos = max((s for _, s in scored), default=0.0)

    print(scored)

    # 4) 임계치: 충분히 유사할 때만 컨텍스트 사용 (속도 최적화)
    THRESH = 0.80
    use_context = best_cos >= THRESH and len(scored) > 0

    context = pack_context([d for d, s in scored if s >= THRESH], token_budget=CTX_TOKEN_BUDGET) if use_context else None
    prompt, max_len = build_prompt(query, use_context, context)
    print(prompt)

    # 5) LLM 생성
    inputs = llm_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_len, padding=False)
    inputs = {k: v.to(llm_model.device) for k, v in inputs.items()}
    with torch.inference_mode():
        out = llm_model.generate(
            **inputs,
            max_new_tokens=dynamic_max_new_tokens(query),
            do_sample=False,
            temperature=0.0,
            top_p=0.1,
            repetition_penalty=1.1,
            eos_token_id=llm_tokenizer.eos_token_id,
            pad_token_id=llm_tokenizer.pad_token_id,
            )
    gen = out[0][inputs["input_ids"].shape[1]:]
    return llm_tokenizer.decode(gen, skip_special_tokens=True).strip()

In [15]:
# ---------------- LLM 로드 & 생성 ----------------
llm_model = AutoModelForCausalLM.from_pretrained(
    LLM_ID,
    device_map="auto",
    load_in_4bit=True,
    torch_dtype=torch.float16
)
if torch.cuda.is_available():
    torch.backends.cuda.matmul.allow_tf32 = True
    try:
        # llm_model.config.attn_implementation = "sdpa"
        llm_model.config.attn_implementation = "flash_attention_2"
        # llm_model.config.attn_implementation = "eager"
    except Exception:
        pass
llm_model.eval()
torch.set_grad_enabled(False)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [16]:
# 후처리 함수
def extract_answer_only(generated_text: str, original_question: str) -> str:
    """
    - "답변:" 이후 텍스트만 추출
    - 객관식 문제면: 정답 숫자만 추출 (실패 시 전체 텍스트 또는 기본값 반환)
    - 주관식 문제면: 전체 텍스트 그대로 반환
    - 공백 또는 빈 응답 방지: 최소 "미응답" 반환
    """
    # "답변:" 기준으로 텍스트 분리
    if "답변:" in generated_text:
        text = generated_text.split("답변:")[-1].strip()
    else:
        text = generated_text.strip()

    # 공백 또는 빈 문자열일 경우 기본값 지정
    if not text:
        return "미응답"

    # 객관식 여부 판단
    is_mc = is_multiple_choice(original_question)

    if is_mc:
        # 숫자만 추출
        match = re.match(r"\D*([1-9][0-9]?)", text)
        if match:
            return match.group(1)
        else:
            # 숫자 추출 실패 시 "0" 반환
            return "0"
    else:
        return text


In [17]:
def strip_explanations(answer: str) -> str:
    """
    키워드 뒤 설명을 제거하고 키워드만 남김
    - '키워드: 설명' → '키워드'
    - '키워드 ... 설명문' → '키워드'
    """
    lines = answer.split("\n")
    cleaned = []
    for line in lines:
        # "키워드: 설명" → "키워드"
        m = re.match(r"^\s*[-0-9.]*\s*([^\:]+)", line)
        if m:
            keyword = m.group(1).strip()
            cleaned.append(keyword)
    return "\n".join(cleaned)

In [18]:
preds = []

for q in tqdm(test['Question'], desc="Inference"):
    print("#################### Question ###########################")
    print(q)
    ans = generate_answer_with_indices(q, indices)
    pred_answer = extract_answer_only(ans, original_question=q)
    print("#################### Answer ###########################")
    print(strip_explanations(pred_answer))
    preds.append(pred_answer)

Inference:   0%|          | 0/16 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


#################### Question ###########################
금융투자업의 종류로 올바르게 나열된 것은?
1 투자매매업, 보험업, 대출업, 자산관리업, 투자일임업, 투자자문업
2 투자중개업, 집합투자업, 신탁업, 대출업, 보험업, 자산관리업
3 투자매매업, 투자중개업, 신탁업, 보험업, 대출업, 자산관리업
4 투자매매업, 투자중개업, 집합투자업, 신탁업, 자산운용업, 금융자문업
5 투자매매업, 투자중개업, 집합투자업, 신탁업, 투자일임업, 투자자문업
[(LawDoc(text='자본시장법 제6조(금융투자업)\n(1) 이 법에서 “금융투자업”이란 이익을 얻을 목적으로 계속적이거나 반복적인 방법으로 행하는 행 위로서 다음 각 호의 어느 하나에 해당하는 업을 말한다. 1. 투자매매업 2. 투자중개업 3. 집합투자업 4. 투자자문업 5. 투자일임업 6. 신탁업 (2) 이 법에서 “투자매매업”이란 누구의 명의로 하든지 자기의 계산으로 금융투자상품의 매도ㆍ매수, 증권의 발행ㆍ 인수 또는 그 청약의 권유, 청약, 청약의 승낙을 영업으로 하는 것을 말한다. (3) 이 법에서 “투자중개업”이란 누구의 명의로 하든지 타인의 계산으로 금융투자상품의 매도ㆍ매수, 그 중개나 청 약의 권유, 청약, 청약의 승낙 또는 증권의 발행ㆍ인수에 대한 청약의 권유, 청약, 청약의 승낙을 영업으로 하는 것을 말한다. (4) 이 법에서 “집합투자업”이란 집합투자를 영업으로 하는 것을 말한다. (5) 제4항에서 “집합투자”란 2인 이상의 투자자로부터 모은 금전등을 투자자로부터 일상적인 운용지시를 받지 아니 하면서 재산적 가치가 있는 투자대상자산을 취득ㆍ처분, 그 밖의 방법으로 운용하고 그 결과를 투자자에게 배분하 여 귀속시키는 것을 말한다. 다만, 다음 각 호의 어느 하나에 해당하는 경우를 제외한다. 1. 대통령령으로 정하는 법률에 따라 사모의 방법으로 금전등을 모아 운용ㆍ배분하는 것으로서 대통령령으로 정하는 투자자의 총수가 대통령령으로 정하는 수 이하인 경우 

Inference:   6%|▋         | 1/16 [00:00<00:05,  2.88it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Inference:  12%|█▎        | 2/16 [00:00<00:03,  4.26it/s]

#################### Answer ###########################
3
#################### Question ###########################
개인정보보호법 제7조의4에 따르면, 위원의 임기는 몇 년이며, 몇 차례 연임할 수 있는가?
1 5년, 두 차례 연임 가능
2 3년, 한 차례 연임 가능
3 2년, 한 차례 연임 가능
4 3년, 연임 제한 없음
5 4년, 두 차례 연임 가능
[(LawDoc(text='개인정보 보호법 제7조의4(위원의 임기)\n(1) 위원의 임기는 3년으로 하되, 한 차례만 연임할 수 있다. (2) 위원이 궐위된 때에는 지체 없이 새로운 위원을 임명 또는 위촉하여야 한다. 이 경우 후임으로 임명 또는 위촉된 위원의 임기는 새로이 개시된다.', meta={'law_id': 'pipa', 'law_name': '개인정보 보호법', 'article_id': '제7조의4', 'article_title': '위원의 임기', 'effective_date': None, 'tok_len': 91, 'source_uri': '../data/개인정보 보호법(법률)(제19234호)(20250313).pdf', 'version': None}), 0.7552537322044373), (LawDoc(text='개인정보 보호법 제7조의5(위원의 신분보장)\n(1) 위원은 다음 각 호의 어느 하나에 해당하는 경우를 제외하고는 그 의사에 반하여 면직 또는 해촉되지 아니한다. 1. 장기간 심신장애로 인하여 직무를 수행할 수 없게 된 경우 2. 제7조의7의 결격사유에 해당하는 경우 3. 이 법 또는 그 밖의 다른 법률에 따른 직무상의 의무를 위반한 경우 (2) 위원은 법률과 양심에 따라 독립적으로 직무를 수행한다.', meta={'law_id': 'pipa', 'law_name': '개인정보 보호법', 'article_id': '제7조의5', 'article_title': '위원의 신분보장', 'effecti

Inference:  19%|█▉        | 3/16 [00:00<00:02,  5.04it/s]

...

#################### Answer ###########################
이 질문에 대한 간결한 답변은 알 수 없습니다.
#################### Question ###########################
SMTP 프로토콜의 보안상 주요 역할을 설명하세요.
[(LawDoc(text='전자금융감독규정 제12조(단말기 보호대책)\n금융회사 또는 전자금융업자는 단말기 보호를 위하여 다음 각 호의 사항을 준수하여야 한 다. 1. 단말기는 인가된 사용자만 사용할 수 있도록 관리할 것 2. 정보처리시스템에 접속하는 단말기에 대해 정당한 사용자인가의 여부를 확인할 수 있는 기록을 유지할 것 7 전자금융감독규정 3. 외부 반출, 인터넷 접속, 그룹웨어 접속의 금지, 정기적인 악성코드 감염여부 확인 등 강화된 보호대책이 적용 되는 중요단말기를 지정할 것 4. 정보유출 및 악성코드 감염 등을 방지할 수 있도록 단말기에서 보조기억매체 등에 접근하는 것을 통제할 것 5. 삭제', meta={'law_id': 'rs_act', 'law_name': '전자금융감독규정', 'article_id': '제12조', 'article_title': '단말기 보호대책', 'effective_date': None, 'tok_len': 204, 'source_uri': '../data/전자금융감독규정(금융위원회고시)(제2025-4호)(20250205).pdf', 'version': None}), 0.6126782894134521), (LawDoc(text='정보통신망법 제57조(통신과금서비스의 안전성 확보 등)\n(1) 통신과금서비스제공자는 통신과금서비스가 안전하게 제공될 수 있도록 선량한 관리자로서의 주의의무를 다하여야 한다. (2) 통신과금서비스제공자는 통신과금서비스를 통한 거래의 안전성과 신뢰성을 확보하기 위하여 대통령령으로 정 하는 바에 따라 업무처리지침의 제정 및 회계처리 구분 등의 관리적 조치와 정보보호시스템 구축 등의 기술적 조치 를 하여야

Inference:  94%|█████████▍| 15/16 [00:17<00:01,  1.69s/it]

#################### Answer ###########################
SMTP(Simple Mail Transfer Protocol) 프로토콜의 보안상 주요 역할은 다음과 같습니다.
인증(Authentication)
암호(Encryption)
인증서(Authorization)
#################### Question ###########################
개인신용평점 하락 가능성과 관련해 설명의무가 발생하는 경우, 금융기관이 설명해야 할 주요 내용은 무엇인가요?
[(LawDoc(text='신용정보법 제35조의2(개인신용평점 하락 가능성 등에 대한 설명의무)\n대통령령으로 정하는 신용정보제공ㆍ이용자는 개인인 신용 정보주체와 신용위험이 따르는 금융거래로서 대통령령으로 정하는 금융거래를 하는 경우 다음 각 호의 사항을 해당 신용정보주체에게 설명하여야 한다. 1. 해당 금융거래로 인하여 개인신용평가회사가 개인신용평점을 만들어 낼 때 해당 신용정보주체에게 불이익이 발 생할 수 있다는 사실 2. 그 밖에 해당 금융거래로 인하여 해당 신용정보주체에게 영향을 미칠 수 있는 사항으로서 대통령령으로 정하는 사항', meta={'law_id': 'ciupa', 'law_name': '신용정보법', 'article_id': '제35조의2', 'article_title': '개인신용평점 하락 가능성 등에 대한 설명의무', 'effective_date': None, 'tok_len': 168, 'source_uri': '../data/신용정보의 이용 및 보호에 관한 법률(법률)(제20304호)(20240814).pdf', 'version': None}), 0.8083800077438354), (LawDoc(text='신용정보법 제36조의2(자동화평가 결과에 대한 설명 및 이의제기 등)\n(1) 개인인 신용정보주체는 개인신용평가회사 및 대통령령으 로 정하는 신용정보제공ㆍ이용자(이하 이 조에서 “개인신용평가회사등”이라 한다)에 대하여 

Inference: 100%|██████████| 16/16 [00:19<00:00,  1.19s/it]

#################### Answer ###########################
금융거래에 따른 개인신용평점 하락 가능성을 설명해야 하는 경우, 금융기관은 다음 사항을 설명해야 합니다.
개인신용평가회사가 개인신용평점을 만들어 낼 때 해당 신용정보주체에게 불이익이 발생할 수 있다는 사실.


In [19]:
# sample_submission = pd.read_csv('../submission/sample_submission.csv')
# sample_submission['Answer'] = preds
# sample_submission.to_csv('../submission/gen_v4.03_submission.csv', index=False, encoding='utf-8-sig')